## Query Post-translational Modifications present in 3D Structures of the PDB.

Post-translational modifications (PTMs) modulate protein function. By mapping the locations of modified amino acid residues onto 3D protein structures, insights into the effect of PTMs can be obtained.

In this notebook PTMs present in PDB structures can be queried by residId, psimodId, uniprotId, and structureChainId. See examples below.

Reference:

BioJava-ModFinder: identification of protein modifications in 3D structures from the Protein Data Bank (2017) Bioinformatics 33: 2047–2049. [doi: doi.org/10.1093/bioinformatics/btx101](https://doi.org/10.1093/bioinformatics/btx101)

In [1]:
import warnings
warnings.filterwarnings("ignore") # numpy version issue?
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_set, collect_list, col, concat_ws
from ipywidgets import interact, IntSlider, widgets
from IPython.display import display
from mmtfPyspark.datasets import pdbToUniProt, pdbPtmDataset
import py3Dmol
import timeit

In [2]:
start_time = timeit.default_timer()

In [3]:
spark = SparkSession.builder.master("local[4]").appName("QueryPdbPtm").getOrCreate()

## Query PTMs by an Identifier

In [4]:
field = widgets.Dropdown(options=['residId', 'psimodId','uniProtId','structureChainId','all'],description='Select field:')
selection = widgets.Textarea(description='Enter id(s):', value='AA0151')

Select query field and enter a comma separated list of ids. Examples:

residId: AA0151 (N4-(N-acetylamino)glucosyl-L-asparagine)

psimodId: MOD:00046, MOD:00047, MOD:00048 (O-phospho-L-serine, O-phospho-L-threonine, O4'-phospho-L-tyrosine)

uniProtId: P13569

structureChainId: 4ZTN.C

In [19]:
display(field)
display(selection)

Dropdown(description='Select field:', index=1, options=('residId', 'psimodId', 'uniProtId', 'structureChainId'…

Textarea(value='MOD:00046, MOD:00047, MOD:00048', description='Enter id(s):')

## Create query string

In [20]:
query = field.value + " IN " + str(selection.value.split(',')).replace("[",'(').replace("]",')').replace(" ", "")
print("Query: " + query)

Query: psimodId IN ('MOD:00046','MOD:00047','MOD:00048')


## Read dataset of PTMs present in PDB

In [21]:
db_ptm = pdbPtmDataset.get_ptm_dataset()
print("Total number of PTM records: ", db_ptm.count())

Total number of PTM records:  1101048


Filter by PTM identifiers

In [22]:
if field.value in ['residId','psimodId']:
    df = db_ptm.filter(query)
    print("Filtered by query:", query)
    print("Number of PTMs matching query:", df.count())
    df.show(5)
else:
    df = db_ptm

Filtered by query: psimodId IN ('MOD:00046','MOD:00047','MOD:00048')
Number of PTMs matching query: 4570
+----------+---------+-------+---------+-------+----+----------------+--------------+
|pdbChainId|pdbResNum|residue| psimodId|residId|ccId|        category|modificationId|
+----------+---------+-------+---------+-------+----+----------------+--------------+
|    1A07.C|      101|    PTR|MOD:00048| AA0039| PTR|modified residue|             8|
|    1A07.D|      101|    PTR|MOD:00048| AA0039| PTR|modified residue|             8|
|    1A1B.C|      101|    PTR|MOD:00048| AA0039| PTR|modified residue|             8|
|    1A1B.D|      101|    PTR|MOD:00048| AA0039| PTR|modified residue|             8|
|    1A1C.C|      101|    PTR|MOD:00048| AA0039| PTR|modified residue|             8|
+----------+---------+-------+---------+-------+----+----------------+--------------+
only showing top 5 rows



## Get PDB to UniProt Residue Mappings

In [23]:
up = pdbToUniProt.get_cached_residue_mappings().filter("pdbResNum IS NOT NULL")
print("Number of PDB to UniProt mappings:", up.count())

Number of PDB to UniProt mappings: 98498569


Filter by UniProtID or structureChainIds

In [24]:
if field.value in ['uniProtId','structureChainId']:
    up = up.filter(query)
    print("Filtered by query: ", query)
    print("Number of records matching query:", up.count())

Find the intersection between the PTM dataset and PDB to UniProt mappings

In [25]:
df = df.withColumnRenamed("pdbResNum","resNum")
st = up.join(df, (up.structureChainId == df.pdbChainId) & (up.pdbResNum == df.resNum)).drop("pdbChainId").drop("resNum")

Show some sample data

In [26]:
hits = st.count()
print("Hits:", hits)
fraction = min(10/hits, 1.0)
st.sample(False, fraction).toPandas().head()

Hits: 4481


,structureChainId,pdbResNum,pdbSeqNum,uniprotId,uniprotNum,residue,psimodId,residId,ccId,category,modificationId
0,3K0K.B,1,1,None,NaN,SEP,MOD:00046,AA0037,SEP,modified residue,6
1,2VAG.A,342,197,P49759,342.0,TPO,MOD:00047,AA0038,TPO,modified residue,7
2,3FBV.J,841,202,P32361,841.0,SEP,MOD:00046,AA0037,SEP,modified residue,6
3,6H0E.J,422,17,P10637,714.0,SEP,MOD:00046,AA0037,SEP,modified residue,6
4,1UU7.A,241,191,O15530,241.0,SEP,MOD:00046,AA0037,SEP,modified residue,6


## Aggregate PTM data by chain-level

In [27]:
st = st.groupBy("structureChainId","uniprotId").agg(collect_list("pdbResNum").alias("pdbResNum"), \
                                                    collect_list("residId").alias("residId"), \
                                                    collect_list("psimodId").alias("psimodId"), \
                                                    collect_list("ccId").alias("ccId"))

Convert aggregated data to Pandas and display some results

In [28]:
pst = st.toPandas()
pst.head(10)

,structureChainId,uniprotId,pdbResNum,residId,psimodId,ccId
0,1P7D.A,P03700,[342],[AA0039],[MOD:00048],[PTR]
1,2R5L.A,A3F9D6,[198],[AA0037],[MOD:00046],[SEP]
2,3D9P.Y,P24928,[2],[AA0037],[MOD:00046],[SEP]
3,4EOP.A,P24941,[160],[AA0038],[MOD:00047],[TPO]
4,4IB1.S,P61925,[621],[AA0037],[MOD:00046],[SEP]
5,4O46.H,Q9YP60,[228],[AA0037],[MOD:00046],[SEP]
6,4YLJ.C,Q13627,[321],[AA0039],[MOD:00048],[PTR]
7,3F5P.G,P08069,"[1161, 1165, 1166]","[AA0039, AA0039, AA0039]","[MOD:00048, MOD:00048, MOD:00048]","[PTR, PTR, PTR]"
8,3ORX.B,O15530,[241],[AA0037],[MOD:00046],[SEP]
9,3Q32.B,O60674,"[1007, 1008]","[AA0039, AA0039]","[MOD:00048, MOD:00048]","[PTR, PTR]"


In [29]:
print("Time to process data:", int(round(timeit.default_timer() - start_time,0)), "seconds")

Time to process data: 1756 seconds


Setup custom visualization

In [30]:
def view_modifications(df, cutoff_distance, *args):

    def view3d(show_labels=True,show_bio_assembly=False, show_surface=False, i=0):
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        res_num = df.iloc[i]['pdbResNum']
        lab1 = df.iloc[i]['residId']
        lab2 = df.iloc[i]['psimodId']
        lab3 = df.iloc[i]['ccId']
        
        # print header
        print ("PDB Id: " + pdb_id + " chain Id: " + chain_id)
        
        # print any specified additional columns from the dataframe
        for a in args:
            if df.iloc[i][a]:
                print(a + ": " + df.iloc[i][a])
        
        mod_res = {'chain': chain_id, 'resi': res_num}  
        
        # select neigboring residues by distance
        surroundings = {'chain': chain_id, 'resi': res_num, 'byres': True, 'expand': cutoff_distance}
        
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly})
    
        # polymer style
        viewer.setStyle({'cartoon': {'color': 'spectrum', 'width': 0.6, 'opacity':0.8}})
        # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # style for modifications
        viewer.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})
        viewer.addStyle(mod_res, {'stick':{'colorscheme':'redCarbon', 'radius': 0.4}})
        viewer.addStyle(mod_res, {'sphere':{'colorscheme':'gray', 'opacity': 0.7}})
        
        # set residue labels    
        if show_labels:
            for residue, l1, l2, l3 in zip(res_num, lab1, lab2, lab3):
                viewer.addLabel(residue + ": " + l1 + " " + l2 + " " + l3, \
                                {'fontColor':'black', 'fontSize': 10, 'backgroundColor': 'lightgray'}, \
                                {'chain': chain_id, 'resi': residue})

        viewer.zoomTo(surroundings)
        
        if show_surface:
            viewer.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'})

        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(df)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_labels=True, show_bio_assembly=False, show_surface=False, i=s_widget)

## Visualize Results
Residues with reported modifications are shown in an all atom prepresentation as red sticks with transparent spheres. Each modified residue position is labeled by the PDB residue number and the type of the modification. Residues surrounding modified residue (within 6 A) are highlighted as yellow sticks. Small molecules within the structure are rendered as gray sticks.

In [31]:
view_modifications(pst, 6, 'uniprotId');

interactive(children=(Checkbox(value=True, description='show_labels'), Checkbox(value=False, description='show…

Most PTMs occur at the protein surface. To visualize the surface, check the show_surface checkbox above.

In [32]:
spark.stop()